# Exercício 3

Os mercados "Deise-Luzia" atendem 11 armazéns de três centros regionais, segundo os
volumes mostrados no quadro a seguir. O custo médio de movimentar bens de um centro
para um armazém é de $0.50 por ton. por km. Ache o programa de transporte ótimo e
seu custo.

Na tabela abaixo, é apresentada a matriz de distância (km) entre centros e armazéns.

|              Centro             	| **W1** 	| **W2** 	| **W3** 	| **W4** 	| **W5** 	| **W6** 	| **W7** 	| **W8** 	| **W9** 	| **W10** 	| **W11** 	| **Capac. centros (kilotons.)** 	|
|:-------------------------------:	|:------:	|:------:	|:------:	|:------:	|:------:	|:------:	|:------:	|:------:	|:------:	|:-------:	|:-------:	|:------------------------------:	|
|              **C1**             	|   10   	|   22   	|   29   	|   45   	|   11   	|   31   	|   42   	|   61   	|   36   	|    21   	|    45   	|               500              	|
|              **C2**             	|   25   	|   35   	|   17   	|   38   	|    9   	|   17   	|   65   	|   45   	|   42   	|    5    	|    41   	|               750              	|
|              **C3**             	|   18   	|   19   	|   22   	|   29   	|   24   	|   54   	|   39   	|   78   	|   51   	|    14   	|    38   	|               400              	|
| **Demanda Armazém (kilotons.)** 	|   112  	|   85   	|   138  	|   146  	|   77   	|   89   	|   101  	|   215  	|   53   	|    49   	|   153   	|                                	|


# Solução

**INSERIR MODELO FORMAL**

In [24]:
# --- Imports --- #
import pyomo.environ as pyo
from pyomo.contrib.latex_printer import latex_printer
from IPython.display import display, Math
import numpy as np
import pandas as pd

In [12]:
# --- Declaração dos dados de entrada --- #
# Matriz de distâncias
custo_tonelada_km = 0.5 # R$/tonelada.km
matriz_distancia = np.array([[10, 22, 29, 45, 11, 31, 42, 61, 36, 21, 45],
                             [25, 35, 17, 38, 9, 17, 65, 45, 42, 5, 41],
                             [18, 19, 22, 29, 24, 54, 39, 78, 51, 14, 38]]) # km
demanda = {'W1': 112000, 
           'W2': 85000, 
           'W3': 138000, 
           'W4': 146000, 
           'W5': 77000, 
           'W6': 89000, 
           'W7': 101000,
           'W8': 215000, 
           'W9': 53000, 
           'W10': 49000, 
           'W11': 153000} # toneladas
oferta = {'C1': 500000, 
          'C2': 750000, 
          'C3': 400000} # toneladas
custo_transporte = {(c, w): matriz_distancia[i, j] * 0.5 for i, c in enumerate(oferta) for j, w in enumerate(demanda)} # r$/tonelada

In [32]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel()

In [33]:
# --- Declaração dos Conjuntos de Iteração --- #
modelo.armazens = pyo.Set(initialize=demanda.keys())
modelo.centros = pyo.Set(initialize=oferta.keys())

In [34]:
# --- Declaração dos Parâmetros do Modelo --- #
modelo.demanda = pyo.Param(modelo.armazens, initialize=demanda) # Demanda de cada armazém
modelo.oferta = pyo.Param(modelo.centros, initialize=oferta) # Oferta de cada centro de distribuição
modelo.custo_transporte = pyo.Param(modelo.centros, modelo.armazens, initialize=custo_transporte) # Custo de transporte de cada centro para cada armazém

In [35]:
# --- Declaração das Variáveis de Decisão --- #
modelo.x = pyo.Var(modelo.centros, modelo.armazens, domain=pyo.NonNegativeIntegers) # Quantidade de toneladas transportadas de cada centro para cada armazém

In [36]:
# --- Declaração da Função Objetivo --- #
def funcao_objetivo(modelo):
    return sum(modelo.custo_transporte[i, j] * modelo.x[i, j]
                for i in modelo.centros
                for j in modelo.armazens
    )
modelo.objetivo = pyo.Objective(rule=funcao_objetivo, sense=pyo.minimize)


In [37]:
# --- Declaração das Restrições --- #
def restricao_demanda(modelo, j):
    '''
    Recebe um objeto de modelo Pyomo e um índice j do conjunto J e retorna a expressão de restrição de demanda
    para o armazém j.
    '''
    return sum(modelo.x[:, j]) == modelo.demanda[j]
# ---
modelo.rest_demanda = pyo.Constraint(modelo.armazens, rule=restricao_demanda)
# --- #
def restricao_oferta(modelo, i):
    '''
    Recebe um objeto de modelo Pyomo e um índice i do conjunto I e retorna a expressão de restrição de oferta
    para o centro i.
    '''
    return sum(modelo.x[i, :]) <= modelo.oferta[i]
# ---
modelo.rest_oferta = pyo.Constraint(modelo.centros, rule=restricao_oferta)

In [38]:
modelo.pprint()

2 Set Declarations
    armazens : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11'}
    centros : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'C1', 'C2', 'C3'}

3 Param Declarations
    custo_transporte : Size=33, Index=centros*armazens, Domain=Any, Default=None, Mutable=False
        Key           : Value
         ('C1', 'W1') :   5.0
        ('C1', 'W10') :  10.5
        ('C1', 'W11') :  22.5
         ('C1', 'W2') :  11.0
         ('C1', 'W3') :  14.5
         ('C1', 'W4') :  22.5
         ('C1', 'W5') :   5.5
         ('C1', 'W6') :  15.5
         ('C1', 'W7') :  21.0
         ('C1', 'W8') :  30.5
         ('C1', 'W9') :  18.0
         ('C2', 'W1') :  12.5
        ('C2', 'W10') :   2.5
        ('C2', 'W11') :  20.5
         ('C2', 'W2') :  17.5
         ('C2', 'W

In [42]:
# --- Declaração do Solver --- #
solver = pyo.SolverFactory('gurobi')
solver.solve(modelo)

{'Problem': [{'Name': 'x1', 'Lower bound': 16678500.0, 'Upper bound': 16678500.0, 'Number of objectives': 1, 'Number of constraints': 14, 'Number of variables': 33, 'Number of binary variables': 0, 'Number of integer variables': 33, 'Number of continuous variables': 0, 'Number of nonzeros': 66, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.003999948501586914', 'Error rc': 0, 'Time': 1.3269295692443848}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [43]:
# --- Extração dos Resultados --- #
dados_transporte = [{'de': i, 'para': j, 'quantidade': val}
                    for (i, j), val in modelo.x.extract_values().items()]
# ---
resultados = pd.DataFrame(dados_transporte).pivot(index='de', columns='para', values='quantidade')
resultados.to_excel('_ex_03_resultados_modelo_matematico.xlsx')

In [44]:
print(f'Função Objetivo: {modelo.objetivo()}') # Valor da função objetivo

Função Objetivo: 16678500.0


In [45]:
dados_transporte

[{'de': 'C1', 'para': 'W1', 'quantidade': 112000.0},
 {'de': 'C1', 'para': 'W2', 'quantidade': 85000.0},
 {'de': 'C1', 'para': 'W3', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W4', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W5', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W6', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W7', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W8', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W9', 'quantidade': 53000.0},
 {'de': 'C1', 'para': 'W10', 'quantidade': -0.0},
 {'de': 'C1', 'para': 'W11', 'quantidade': -0.0},
 {'de': 'C2', 'para': 'W1', 'quantidade': -0.0},
 {'de': 'C2', 'para': 'W2', 'quantidade': -0.0},
 {'de': 'C2', 'para': 'W3', 'quantidade': 138000.0},
 {'de': 'C2', 'para': 'W4', 'quantidade': -0.0},
 {'de': 'C2', 'para': 'W5', 'quantidade': 77000.0},
 {'de': 'C2', 'para': 'W6', 'quantidade': 89000.0},
 {'de': 'C2', 'para': 'W7', 'quantidade': -0.0},
 {'de': 'C2', 'para': 'W8', 'quantidade': 215000.0},
 {'de': 'C2', 'para': 'W9', 'quantidade': -